### Imports

In [1]:
import numpy as np
from typing import Tuple

from tsc_utils.utilities import get_bravais, get_pauli, Fermi, get_RPTS, get_KPTS
from tsc_utils.basis_vectors import single_atom

### Config constants

### Get basis vectors

### Get other constants based on config constants

In [32]:
# Get the Pauli matrices
s_0, s_1, s_2, s_3 = get_pauli()

# Get lattice vectors through Bravais vectors
a_1, a_2, a_3 = ALAT*a_1, ALAT*a_2, ALAT*a_3


# Get NUMK
NUMK: int = N_x * N_y * N_z

# Get 

In [17]:
ALAT = 2.0 # lattice constant
a_1, a_2, a_3 = get_vectors("BCC")

In [20]:
a_1

array([ 1.,  1., -1.])

In [8]:
import numpy as np
import timeit

# Loop-based implementation
def create_slab_loop(start_index, end_index, fixed_coord_1, fixed_coord_2, axis, atom_type, E_0, U, n_0, B_x, B_y, B_z, V):
    assert axis in ('x', 'y', 'z'), "Invalid axis. Choose from 'x', 'y', or 'z'"
    num_atoms = end_index - start_index + 1
    slab = np.zeros((num_atoms, 11))
    for i in range(num_atoms):
        if axis == 'x':
            coords = [start_index + i, fixed_coord_1, fixed_coord_2]
        elif axis == 'y':
            coords = [fixed_coord_1, start_index + i, fixed_coord_2]
        else:
            coords = [fixed_coord_1, fixed_coord_2, start_index + i]
        atom = np.array(coords + [atom_type, E_0, U, n_0, B_x, B_y, B_z, V])
        slab[i] = atom
    return slab

# Vectorized implementation
def create_slab_vectorized(start_index, end_index, fixed_coord_1, fixed_coord_2, axis, atom_type, E_0, U, n_0, B_x, B_y, B_z, V):
    assert axis in ('x', 'y', 'z'), "Invalid axis. Choose from 'x', 'y', or 'z'"
    num_atoms = end_index - start_index + 1
    varying_coord = np.arange(start_index, end_index + 1).reshape(-1, 1)
    fixed_coords_1 = np.full((num_atoms, 1), fixed_coord_1)
    fixed_coords_2 = np.full((num_atoms, 1), fixed_coord_2)
    if axis == 'x':
        coords = np.hstack((varying_coord, fixed_coords_1, fixed_coords_2))
    elif axis == 'y':
        coords = np.hstack((fixed_coords_1, varying_coord, fixed_coords_2))
    else:
        coords = np.hstack((fixed_coords_1, fixed_coords_2, varying_coord))
    other_params = np.array([atom_type, E_0, U, n_0, B_x, B_y, B_z, V])
    other_params_repeated = np.tile(other_params, (num_atoms, 1))
    slab = np.hstack((coords, other_params_repeated))
    return slab

# Time the implementations
loop_time = timeit.timeit(
    stmt="create_slab_loop(0, 49, 0.0, 0.0, 'z', 1, 0.0, 0.0, 0.4, 0.0, 0.0, 0.0, 0.5)",
    setup="from __main__ import create_slab_loop",
    number=1000  # Number of repetitions
)

vectorized_time = timeit.timeit(
    stmt="create_slab_vectorized(0, 49, 0.0, 0.0, 'z', 1, 0.0, 0.0, 0.4, 0.0, 0.0, 0.0, 0.5)",
    setup="from __main__ import create_slab_vectorized",
    number=1000  # Number of repetitions
)

# Print the results
print(f"Loop-based time: {loop_time:.6f} seconds")
print(f"Vectorized time: {vectorized_time:.6f} seconds")


Loop-based time: 0.060377 seconds
Vectorized time: 0.013274 seconds


In [27]:
# NEEDS FIXING
def get_NumDen(EIGENVALUES: np.ndarray, EIGENVECTORS: np.ndarray, NUMT: int, NUMK: int, 
            PI: float, NUME: int, lorentzbroad: float, metalorno: int) -> None:
    
    numdensityperatom = np.zeros((NUMT + 1, NUME))
    numdensity = np.zeros((2, NUME))
    
    energyintervals = (np.max(EIGENVALUES) - np.min(EIGENVALUES)) / (NUME - 1)
    
    # These are the energies E
    energies = np.min(EIGENVALUES) + energyintervals * np.arange(NUME)
    numdensityperatom[0, :] = energies
    numdensity[0, :] = energies
    numdensity[1, :] = 0.0
    
    # Calculation of the DoS PER ATOM
    term1 = np.abs(EIGENVECTORS[:NUMT, :])**2
    term2 = np.abs(EIGENVECTORS[NUMT:, :])**2
    terms_sum = term1 + term2
    
    # Reshape terms_sum to have an extra dimension for NUME
    terms_sum_reshaped = terms_sum[:, np.newaxis, :]
    
    # Reshape diff_square to have an extra dimension for NUMT
    diff_square = (energies[:, np.newaxis] - EIGENVALUES)**2
    diff_square_reshaped = diff_square[np.newaxis, :, :]
    
    # Now, both arrays have a shape of (NUMT, NUME, 4*NUMT*NUMK)
    dos_terms = (lorentzbroad / PI) * terms_sum_reshaped / (diff_square_reshaped + lorentzbroad**2)
    
    numdensityperatom[1:, :] = np.sum(dos_terms, axis=2)
    
    # Normalization
    numdensityperatom[1:, :] /= NUMK
    
    # File writing
    if metalorno == 0:
        np.savetxt('numdensityperatom2.txt', numdensityperatom.T, fmt='%17.8f')
        numdensity[1, :] = np.mean(numdensityperatom[1:, :], axis=0)
        np.savetxt('numdensity2.txt', numdensity.T, fmt='%17.8f')
    elif metalorno == 1:
        np.savetxt('metalnumdensityperatom2.txt', numdensityperatom.T, fmt='%17.8f')
        numdensity[1, :] = np.mean(numdensityperatom[1:, :], axis=0)
        np.savetxt('metalnumdensity2.txt', numdensity.T, fmt='%17.8f')


In [8]:
a_1, a_2, a_3 = get_vectors(vec_type="SC")

RPTS = get_RPTS(a_1, a_2, a_3, 10, 10000)

In [9]:
RPTS.shape

(3, 9261)